<a href="https://colab.research.google.com/github/pcw11/bigdataprograming-gj/blob/main/%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%9D%91%EC%9A%A9%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%98%EB%B0%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 데이터 전처리

In [ ]:
import pandas as pd

# 1. 원본 데이터 생성 (이전에 제공한 표 2, 3의 데이터)

# --- 표 3: 온라인 동영상 서비스(OTT) 이용률 (전체) ---
data_utilization = {
    '연령대': ['전체', '10대', '20대', '30대', '40대', '50대', '60대 이상', '70대 이상'],
    '2022년': [72.0, 90.0, 97.5, 95.7, 86.9, 70.2, 54.4, 19.3],
    '2023년': [77.0, 90.9, 97.5, 95.7, 88.5, 75.9, 55.6, 22.3],
    '2024년': [79.2, 92.7, 97.8, 96.0, 90.8, 81.0, 66.7, 27.1]
}
df_utilization = pd.DataFrame(data_utilization)

# --- 표 2: OTT 유료 이용 비율 (전체 이용자 중) ---
data_paid_ratio = {
    '연령대': ['전체', '10대', '20대', '30대', '40대', '50대', '60대', '70대 이상'],
    '2022년': [35.9, 68.1, 79.4, 79.0, 82.6, 61.1, 25.8, 7.2],
    '2023년': [37.9, 68.7, 81.8, 79.2, 82.9, 51.1, 28.6, 12.0],
    '2024년': [39.0, 68.9, 82.8, 80.6, 83.6, 51.1, 29.3, 12.6]
}
df_paid_ratio = pd.DataFrame(data_paid_ratio)

# ----------------------------------------------------
# 2. 데이터 변환 및 통합 (Wide -> Long Format & Merge)
# ----------------------------------------------------

def transform_to_long_format(df, value_name, variable_name='연도'):
    """Wide Format DataFrame을 Long Format으로 변환하는 함수"""
    # '연령대'를 고정하고 나머지 컬럼(연도)을 행으로 피봇
    df_long = pd.melt(df, id_vars=['연령대'],
                      var_name=variable_name,
                      value_name=value_name)
    # 연도 컬럼의 문자열 '년' 제거 후 정수형으로 변환
    df_long[variable_name] = df_long[variable_name].str.replace('년', '').astype(int)
    return df_long

# 2.1. OTT 전체 이용률 데이터 변환
df_util_long = transform_to_long_format(df_utilization, '전체_OTT_이용률')

# 2.2. 유료 OTT 이용 비율 데이터 변환
df_paid_long = transform_to_long_format(df_paid_ratio, '유료_OTT_이용비율')

# 2.3. 두 데이터셋을 '연령대'와 '연도'를 기준으로 통합 (Merge)
df_merged = pd.merge(df_util_long, df_paid_long,
                     on=['연령대', '연도'],
                     how='inner')

print("### 2. 통합 및 Long Format 변환 결과 (일부) ###")
print(df_merged.head(10))

# ----------------------------------------------------
# 3. 특성 생성 (Feature Engineering)
# ----------------------------------------------------

# 3.1. 유료 전환율 (Conversion Rate) 파생 변수 생성
# 계산: (유료 OTT 이용 비율 / 전체 OTT 이용률) * 100
df_merged['유료_전환율'] = (df_merged['유료_OTT_이용비율'] / df_merged['전체_OTT_이용률']) * 100

# 3.2. 연도별 이용률 성장률 파생 변수 생성 (전년 대비 성장률)
# 성장률 계산은 '연령대' 그룹별로 연도 순서대로 계산해야 함
df_merged['성장률'] = df_merged.groupby('연령대')['전체_OTT_이용률'].pct_change() * 100

# ----------------------------------------------------
# 4. 최종 데이터 정리 (Final Touches)
# ----------------------------------------------------

# '전체' 연령대 데이터는 분석 시 방해가 될 수 있으므로 제거 (옵션)
df_final = df_merged[df_merged['연령대'] != '전체'].copy()

# '60대 이상'과 '70대 이상'을 '60대 이상'으로 통일하는 작업 등을 추가할 수 있으나,
# 원본 데이터의 구조를 유지하기 위해 생략.

print("\n### 3. 특성 생성 및 최종 전처리 결과 (일부) ###")
print(df_final.sort_values(by=['연령대', '연도']).head(10))

### 2. 통합 및 Long Format 변환 결과 (일부) ###
      연령대    연도  전체_OTT_이용률  유료_OTT_이용비율
0      전체  2022        72.0         35.9
1     10대  2022        90.0         68.1
2     20대  2022        97.5         79.4
3     30대  2022        95.7         79.0
4     40대  2022        86.9         82.6
5     50대  2022        70.2         61.1
6  70대 이상  2022        19.3          7.2
7      전체  2023        77.0         37.9
8     10대  2023        90.9         68.7
9     20대  2023        97.5         81.8

### 3. 특성 생성 및 최종 전처리 결과 (일부) ###
    연령대    연도  전체_OTT_이용률  유료_OTT_이용비율     유료_전환율       성장률
1   10대  2022        90.0         68.1  75.666667       NaN
8   10대  2023        90.9         68.7  75.577558  1.000000
15  10대  2024        92.7         68.9  74.325782  1.980198
2   20대  2022        97.5         79.4  81.435897       NaN
9   20대  2023        97.5         81.8  83.897436  0.000000
16  20대  2024        97.8         82.8  84.662577  0.307692
3   30대  2022        95.7         79.0  82.549634      

In [ ]:
import pandas as pd

# 1. 원본 데이터 생성 (이전에 제공한 표 4의 데이터)
data_platform = {
    '플랫폼': ['유튜브', '넷플릭스', '티빙', '웨이브', '쿠팡플레이', '디즈니플러스', '왓챠', '네이버', '카카오', '기타'],
    '2022년': [71.0, 25.0, 5.2, 9.7, 3.6, 3.8, 2.2, 1.4, 1.4, 1.2],
    '2023년': [72.7, 31.5, 6.5, 5.2, 5.9, 3.6, 1.8, 1.7, 1.5, 1.0],
    '2024년': [69.3, 36.0, 8.3, 5.9, 6.1, 3.9, 1.5, 1.8, 1.2, 0.6]
}
df_platform = pd.DataFrame(data_platform)

# ----------------------------------------------------
# 2. 데이터 변환 (Wide -> Long Format)
# ----------------------------------------------------

# '플랫폼'을 고정하고, 연도별 데이터를 행으로 변환합니다.
df_platform_long = pd.melt(df_platform,
                           id_vars=['플랫폼'],
                           var_name='연도',
                           value_name='이용률')

# 연도 컬럼의 문자열 '년' 제거 후 정수형으로 변환
df_platform_long['연도'] = df_platform_long['연도'].str.replace('년', '').astype(int)

# ----------------------------------------------------
# 3. 특성 생성 (Feature Engineering)
# ----------------------------------------------------

# 3.1. 연도별 이용률 성장률 파생 변수 생성 (전년 대비 성장률)
# 성장률 계산은 '플랫폼' 그룹별로 연도 순서대로 계산해야 함
df_platform_long['성장률'] = df_platform_long.groupby('플랫폼')['이용률'].pct_change() * 100

# ----------------------------------------------------
# 4. 최종 데이터 정리 및 인코딩
# ----------------------------------------------------

# 4.1. 플랫폼 순위 파악을 위한 데이터 정렬
df_platform_final = df_platform_long.sort_values(by=['연도', '이용률'], ascending=[True, False]).copy()

# 4.2. 범주형 변수 처리 (선택적: 머신러닝 모델에 사용 시)
# One-Hot Encoding을 통해 플랫폼 이름을 이진 특성으로 변환하여 모델 입력에 사용할 수 있습니다.
# (현재 단계에서는 시각화/분석에 용이하도록 Long Format을 유지하고, 필요 시 아래 코드를 사용합니다.)
# df_encoded = pd.get_dummies(df_platform_long, columns=['플랫폼'], prefix='is')

# ----------------------------------------------------
# 5. 결과 출력
# ----------------------------------------------------
print("### 2024년 플랫폼 이용률 기준 상위 5개 플랫폼 ###")
print(df_platform_final[df_platform_final['연도'] == 2024].head(5))

print("\n### 최종 전처리 결과 (성장률 포함, 넷플릭스 데이터) ###")
print(df_platform_final[df_platform_final['플랫폼'] == '넷플릭스'])

### 2024년 플랫폼 이용률 기준 상위 5개 플랫폼 ###
      플랫폼    연도   이용률        성장률
20    유튜브  2024  69.3  -4.676754
21   넷플릭스  2024  36.0  14.285714
22     티빙  2024   8.3  27.692308
24  쿠팡플레이  2024   6.1   3.389831
23    웨이브  2024   5.9  13.461538

### 최종 전처리 결과 (성장률 포함, 넷플릭스 데이터) ###
     플랫폼    연도   이용률        성장률
1   넷플릭스  2022  25.0        NaN
11  넷플릭스  2023  31.5  26.000000
21  넷플릭스  2024  36.0  14.285714


In [ ]:
import pandas as pd
import io

# 1. 원본 데이터 생성 (표 2, 3)

# --- 표 2: 콘텐츠 선택 기준 (1) ---
data_criteria_1_str = """
구분,전체 (N=2,697),사후수정 및 전체 시청 가능,완성도 높은 콘텐츠,미디어에서 화제성,재방송 혹은 회차 몰아보기,매체/연령 적합성,광고/미디어 추천,주변 지인 의견/추천
전체,77.0,38.6,36.4,35.6,34.8,34.2,32.7,31.5
남성,1250,76.4,38.9,36.1,32.6,32.7,31.5,32.6,31.5
여성,1437,77.5,38.4,36.6,38.2,36.6,34.2,33.8,31.5
10대,318,76.1,35.7,32.5,37.6,40.2,41.0,37.9,36.6
20대,472,76.8,38.2,34.1,39.1,35.0,26.2,26.2,26.2
30대,616,76.6,34.8,41.8,31.8,36.3,33.1,33.1,33.1
40대,629,78.6,43.5,37.5,37.3,35.1,35.9,35.9,35.9
50대,434,76.5,37.6,33.9,33.9,32.0,34.3,34.3,34.3
60대,190,74.1,37.7,34.8,26.8,26.5,42.9,42.9,42.9
70대 이상,28,93.9,73.6,26.9,23.3,27.8,21.1,21.1,21.1
"""
df_criteria_1 = pd.read_csv(io.StringIO(data_criteria_1_str), sep=',')
df_criteria_1 = df_criteria_1.iloc[:, [0] + list(range(2, df_criteria_1.shape[1]))] # N 컬럼 제거


# --- 표 3: 콘텐츠 선택 기준 (2) ---
data_criteria_2_str = """
구분,전체 (N=2,697),콘텐츠 인기 순위,콘텐츠 리뷰,TV, 잡지 등 통계/정보,원작 (웹툰, 소설, 제작물 등),사후수정 스타일 선호,사후수정 시청 방식 선호
전체,33.3,31.6,24.9,20.6,6.4,6.4
남성,1250,32.8,31.0,24.4,20.4,6.4,4.0
여성,1437,33.7,32.0,25.4,20.7,6.5,8.5
10대,318,34.4,32.6,24.9,21.7,7.9,6.7
20대,472,34.9,39.3,22.6,24.4,6.7,7.6
30대,616,36.5,32.5,24.3,24.7,6.7,7.2
40대,629,34.9,33.4,23.8,17.7,7.8,6.2
50대,434,29.8,26.3,29.1,15.0,5.6,5.7
60대,190,22.9,18.4,30.3,20.4,0.7,3.8
70대 이상,28,9.3,0.0,0.0,5.8,0.0,0.0
"""
df_criteria_2 = pd.read_csv(io.StringIO(data_criteria_2_str), sep=',')
df_criteria_2 = df_criteria_2.iloc[:, [0] + list(range(2, df_criteria_2.shape[1]))] # N 컬럼 제거


# ----------------------------------------------------
# 2. 데이터 변환 (Wide -> Long Format)
# ----------------------------------------------------

def transform_criteria(df, base_columns, value_name='응답률'):
    """콘텐츠 선택 기준 데이터를 Long Format으로 변환하는 함수"""
    # '구분'을 고정하고 나머지 컬럼(기준)을 행으로 피봇
    df_long = pd.melt(df,
                      id_vars=base_columns,
                      var_name='선택_기준',
                      value_name=value_name)
    return df_long

# 두 데이터셋 모두 Long Format으로 변환
df_long_1 = transform_criteria(df_criteria_1, base_columns=['구분'])
df_long_2 = transform_criteria(df_criteria_2, base_columns=['구분'])

# 두 개의 기준 데이터셋을 수직으로 통합 (Concatenate)
df_combined = pd.concat([df_long_1, df_long_2], ignore_index=True)

# ----------------------------------------------------
# 3. 특성 생성: 연령대와 성별 분리 및 정제
# ----------------------------------------------------

def classify_group(group):
    """'구분' 컬럼을 '성별'과 '연령대'로 분리하는 함수 (이전과 동일)"""
    if group in ['남성', '여성']:
        return group, '전체'
    elif group.endswith('대') or group == '70대 이상':
        return '전체', group
    return '전체', '전체'

df_combined[['성별', '연령대']] = df_combined['구분'].apply(classify_group, result_type='expand')
df_combined.drop('구분', axis=1, inplace=True) # 원본 '구분' 컬럼 삭제

# ----------------------------------------------------
# 4. 최종 데이터 정리 및 출력
# ----------------------------------------------------

# '전체' 행을 제외하고 응답률이 높은 순서로 정렬
df_final_criteria = df_combined[
    (df_combined['성별'] != '전체') | (df_combined['연령대'] != '전체')
].sort_values(by=['성별', '연령대', '응답률'], ascending=[True, True, False])

print("### OTT 콘텐츠 선택 기준 최종 전처리 결과 (일부) ###")
# 20대 여성의 선택 기준 상위 항목을 예시로 출력
print(df_final_criteria[
    (df_final_criteria['성별'] == '여성') & (df_final_criteria['연령대'] == '전체')
].head(5))

print("\n### 40대 남성의 콘텐츠 선택 기준 상위 5개 항목 ###")
# 40대(연령대 전체)의 데이터에서 상위 5개 항목 출력
print(df_final_criteria[
    (df_final_criteria['연령대'] == '40대')
].sort_values(by='응답률', ascending=False).head(5))

In [ ]:
import pandas as pd

# 1. 원본 데이터 정의 (핵심 지표: 5점 평균, 100점 평균)
# 5점 평균만 사용하고, 나머지는 분석의 단순화를 위해 제외합니다.

def create_df_from_data(data, content_type):
    """원시 데이터에서 '구분', '5점 평균'을 추출하여 DataFrame을 생성합니다."""
    # 데이터는 5점 평균이 9번째 컬럼, 100점 평균이 10번째 컬럼에 위치한다고 가정
    # 컬럼 인덱스: 0(구분), 9(5점 평균), 10(100점 평균)
    df = pd.DataFrame(data)
    df = df.iloc[:, [0, 9, 10]].copy()
    df.columns = ['구분', '5점_평균', '100점_평균']
    df['콘텐츠_종류'] = content_type
    return df

# 오리지널 콘텐츠 데이터
data_original = {
    0: ['전체', '남성', '여성', '10대', '20대', '30대', '40대', '50대', '60대', '70대 이상'],
    9: [3.97, 3.90, 4.04, 3.96, 4.10, 3.95, 3.87, 3.96, 3.79, 3.57],
    10: [79.5, 77.9, 80.8, 79.6, 82.0, 79.0, 77.5, 79.1, 75.7, 75.7]
}
df_orig = create_df_from_data(data_original, '오리지널')

# 비독점 콘텐츠 데이터
data_beO = {
    0: ['전체', '남성', '여성', '10대', '20대', '30대', '40대', '50대', '60대', '70대 이상'],
    9: [3.71, 3.66, 3.76, 3.66, 3.75, 3.20, 3.74, 3.64, 3.82, 3.69],
    10: [74.2, 73.1, 75.2, 73.3, 75.1, 64.3, 74.8, 72.7, 72.5, 73.8]
}
df_beO = create_df_from_data(data_beO, '비독점')

# 독점 콘텐츠 데이터
data_Own = {
    0: ['전체', '남성', '여성', '10대', '20대', '30대', '40대', '50대', '60대', '70대 이상'],
    9: [3.73, 3.72, 3.74, 3.74, 3.84, 3.75, 3.71, 3.71, 3.55, 3.52],
    10: [74.6, 74.3, 74.9, 74.7, 76.7, 75.0, 74.2, 74.1, 70.9, 70.4]
}
df_Own = create_df_from_data(data_Own, '독점')

# 2. 모든 데이터셋 통합 (Concatenate)
df_combined = pd.concat([df_orig, df_beO, df_Own], ignore_index=True)

# 3. 특성 생성: 연령대와 성별 분리 및 정제
def classify_group(group):
    """'구분' 컬럼을 '성별'과 '연령대'로 분리하는 함수"""
    if group in ['남성', '여성']:
        return group, '전체'
    elif group.endswith('대') or group == '70대 이상':
        return '전체', group
    return '전체', '전체'

df_combined[['성별', '연령대']] = df_combined['구분'].apply(classify_group, result_type='expand')
df_combined.drop('구분', axis=1, inplace=True) # 원본 '구분' 컬럼 삭제

# 4. 최종 데이터 정리 및 출력
# '전체' 그룹을 제외하고 '콘텐츠_종류' 및 '연령대'별로 정렬
df_final_preference = df_combined[
    (df_combined['성별'] != '전체') | (df_combined['연령대'] != '전체')
].sort_values(by=['콘텐츠_종류', '연령대', '5점_평균'], ascending=[True, True, False])

print("### 콘텐츠 종류별 선호도 최종 전처리 결과 (일부) ###")
print(df_final_preference.head(15))

IndexError: positional indexers are out-of-bounds

### 분석 코드 및 그래프

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Matplotlib 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

def plot_utilization_growth_rate(df):
    plt.figure(figsize=(12, 7))

    sns.barplot(
        data=df,
        x='연령대',
        y='성장률',
        hue='연도',
        palette='rocket'
    )

    plt.title('OTT 이용률 성장률 (전년 대비) vs. 연령대별 비교', fontsize=16, pad=20)
    plt.ylabel('OTT 이용률 성장률 (%)', fontsize=12)
    plt.xlabel('연령대', fontsize=12)
    plt.ylim(0, df['성장률'].max() * 1.2)
    plt.legend(title='연도', loc='upper right')

    # 데이터 레이블 추가
    ax = plt.gca()
    for p in ax.patches:
        height = p.get_height()
        if not np.isnan(height) and abs(height) > 0.1:
            ax.annotate(f'{height:.2f}',
                        (p.get_x() + p.get_width() / 2., height),
                        ha='center', va='center',
                        xytext=(0, 9),
                        textcoords='offset points', fontsize=9)

    plt.show()



# =====================================================================
# 📌 데이터 생성
# =====================================================================

data_util = {
    '연령대': [
        '10대','10대','10대','20대','20대','20대','30대','30대','30대',
        '40대','40대','40대','50대','50대','50대','60대 이상','60대 이상','60대 이상',
        '70대 이상','70대 이상','70대 이상'
    ],
    '연도': [
        2022,2023,2024,2022,2023,2024,2022,2023,2024,
        2022,2023,2024,2022,2023,2024,2022,2023,2024,
        2022,2023,2024
    ],
    '전체_OTT_이용률': [
        90.0,90.9,92.7,97.5,97.5,97.8,95.7,95.7,96.0,
        86.9,88.5,90.8,70.2,75.9,81.0,54.4,55.6,66.7,
        19.3,22.3,27.1
    ]
}

df_merged = pd.DataFrame(data_util)

# =====================================================================
# 📌 성장률 계산
# =====================================================================
df_merged['성장률'] = df_merged.groupby('연령대')['전체_OTT_이용률'].pct_change() * 100
df_merged.replace([np.inf, -np.inf], np.nan, inplace=True)

# 2022년 데이터는 성장률 없음 → 제외
df_growth = df_merged[df_merged['연도'] >= 2023].copy()

# =====================================================================
# 📌 그래프 실행
# =====================================================================
plot_utilization_growth_rate(df_growth)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Matplotlib 설정
plt.rcParams['font.family'] = 'Malgun Gothic' # 한글 폰트 설정 (Windows 기준)
# plt.rcParams['font.family'] = 'AppleGothic' # Mac 사용자용
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

# --- 데이터 생성 (이전에 전처리된 데이터 재현) ---
data_util = {
    '연령대': ['10대', '10대', '10대', '20대', '20대', '20대', '30대', '30대', '30대', '40대', '40대', '40대', '50대', '50대', '50대', '60대 이상', '60대 이상', '60대 이상', '70대 이상', '70대 이상', '70대 이상'],
    '연도': [2022, 2023, 2024, 2022, 2023, 2024, 2022, 2023, 2024, 2022, 2023, 2024, 2022, 2023, 2024, 2022, 2023, 2024, 2022, 2023, 2024],
    # '전체_OTT_이용률' 컬럼은 생략
    '유료_전환율': [75.67, 75.58, 74.33, 81.44, 83.90, 84.66, 82.55, 82.76, 83.96, 95.05, 93.67, 92.07, 87.04, 67.32, 63.09, 47.43, 51.44, 43.93, 37.31, 53.81, 46.50]
}
df_merged = pd.DataFrame(data_util)def plot_paid_conversion_rate_trend(df):
    plt.figure(figsize=(10, 6))

    # Line Plot: 연령대별 유료 전환율 추이 (연도별)
    ax = sns.lineplot(
        data=df,
        x='연도',
        y='유료_전환율',
        hue='연령대',
        marker='o',
        linewidth=2.5,
        palette='Spectral_r'
    )

    plt.title('연령대별 OTT 이용자의 유료 전환율 변화 추이 (2022년~2024년)', fontsize=16, pad=20)
    plt.ylabel('유료 전환율 (%)', fontsize=12)
    plt.xlabel('연도', fontsize=12)
    plt.ylim(30, 100)
    plt.xticks([2022, 2023, 2024])
    plt.legend(title='연령대', loc='center left', bbox_to_anchor=(1, 0.5))

    # 2024년 데이터 포인트에 레이블 추가
    df_2024 = df[df['연도'] == 2024]
    for index, row in df_2024.iterrows():
        ax.text(row['연도'] + 0.05, row['유료_전환율'],
                f"{row['유료_전환율']:.1f}",
                ha='left', va='center', fontsize=9)

    plt.show()

# 그래프 실행
plot_paid_conversion_rate_trend(df_merged)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Matplotlib 설정
plt.rcParams['font.family'] = 'Malgun Gothic' # 한글 폰트 설정 (Windows 기준)
# plt.rcParams['font.family'] = 'AppleGothic' # Mac 사용자용
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

# --- OTT 서비스 구독 이유 데이터 생성 (이전에 전처리된 데이터 재현) ---
# Long format으로 전처리된 데이터 (연령대별 응답률)의 핵심 요인만 발췌하여 재현
data_reasons = {
    '연령대': ['10대', '20대', '30대', '40대', '50대', '60대', '70대 이상'] * 5,
    '이용_이유': ['풍부한 콘텐츠 다양성'] * 7 + ['시간 제약 없이 재시청'] * 7 + ['OTT에서만 제공하는 콘텐츠'] * 7 + ['프로모션 할인 제공'] * 7 + ['친구/지인 추천'] * 7,
    '응답률': [68.7, 68.5, 69.2, 69.9, 66.3, 65.6, 44.3,  # 풍부한 콘텐츠 다양성
              61.6, 61.8, 60.1, 57.4, 53.6, 38.2, 48.8,  # 시간 제약 없이 재시청
              32.9, 34.0, 31.2, 36.5, 29.7, 21.9, 34.2,  # OTT에서만 제공하는 콘텐츠
              15.8, 10.0, 13.5, 13.9, 11.8, 10.8, 0.0,   # 프로모션 할인 제공
              9.5, 16.2, 11.5, 10.7, 7.7, 12.0, 13.9]    # 친구/지인 추천
}
df_reasons = pd.DataFrame(data_reasons)

# 피벗 테이블 생성: 히트맵을 위한 Wide Format으로 변환
df_pivot = df_reasons.pivot(
    index='이용_이유',
    columns='연령대',
    values='응답률'
)
# 연령대 순서 정의
ordered_columns = ['10대', '20대', '30대', '40대', '50대', '60대', '70대 이상']
df_pivot = df_pivot[ordered_columns]def plot_subscription_reason_heatmap(df_pivot):
    plt.figure(figsize=(12, 8))

    # Heatmap 생성
    sns.heatmap(
        df_pivot,
        annot=True, # 셀 안에 응답률 값 표시
        fmt=".1f",
        cmap="YlGnBu", # 색상 팔레트
        linewidths=.5, # 셀 경계선 두께
        cbar_kws={'label': '응답률 (%)'} # 컬러바 레이블
    )

    plt.title('연령대별 OTT 서비스 구독을 유도하는 주요 요인 (응답률 히트맵)', fontsize=16, pad=20)
    plt.xlabel('연령대', fontsize=12)
    plt.ylabel('OTT 서비스 구독 이유', fontsize=12)
    plt.yticks(rotation=0) # y축 레이블 수평으로 표시
    plt.show()

# 그래프 실행
plot_subscription_reason_heatmap(df_pivot)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Matplotlib 설정
plt.rcParams['font.family'] = 'Malgun Gothic' # 한글 폰트 설정 (Windows 기준)
# plt.rcParams['font.family'] = 'AppleGothic' # Mac 사용자용
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

# --- 콘텐츠 종류별 선호도 데이터 생성 (이전에 전처리된 데이터 재현) ---
# Long format으로 전처리된 데이터 (연령대별 5점 평균)
data_pref = {
    '연령대': ['10대', '20대', '30대', '40대', '50대', '60대', '70대 이상'] * 3,
    '콘텐츠_종류': ['오리지널'] * 7 + ['비독점'] * 7 + ['독점'] * 7,
    '5점_평균': [3.96, 4.10, 3.95, 3.87, 3.96, 3.79, 3.57, # 오리지널
                3.66, 3.75, 3.20, 3.74, 3.64, 3.82, 3.69, # 비독점
                3.74, 3.84, 3.75, 3.71, 3.71, 3.55, 3.52]  # 독점
}
df_content_pref = pd.DataFrame(data_pref)

# 오리지널 콘텐츠 선호도 순서대로 연령대를 정렬하기 위한 기준 생성
df_pivot = df_content_pref[df_content_pref['콘텐츠_종류'] == '오리지널'].sort_values(by='5점_평균', ascending=False)
order = df_pivot['연령대'].tolist()def plot_content_preference_comparison(df, order):
    plt.figure(figsize=(12, 7))

    # Point Plot: 연령대별 콘텐츠 선호도 비교
    ax = sns.pointplot(
        data=df,
        x='연령대',
        y='5점_평균',
        hue='콘텐츠_종류',
        order=order, # 오리지널 선호도 순으로 연령대 정렬
        palette={'오리지널': 'darkblue', '비독점': 'orange', '독점': 'green'},
        markers=['o', 's', '^'],
        linestyles=['-', '--', '-.'],
        dodge=True, # 그룹 간의 위치를 살짝 이동하여 겹침 방지
    )

    plt.title('연령대별 콘텐츠 종류별 선호도 비교 (5점 평균)', fontsize=16, pad=20)
    plt.xlabel('연령대 (오리지널 선호도 순 정렬)', fontsize=12)
    plt.ylabel('선호도 (5점 평균)', fontsize=12)
    plt.legend(title='콘텐츠 종류', loc='upper right')
    plt.ylim(3.0, 4.3) # 선호도 점수 범위 명확히 표시

    # 데이터 레이블 추가 (20대, 30대, 40대만 주요 포인트 레이블링)
    target_ages = ['20대', '30대', '40대']

    for line in ax.lines:
        for x, y, label in zip(line.get_xdata(), line.get_ydata(), order):
            if label in target_ages:
                 ax.text(x, y + 0.05, f"{y:.2f}", color=line.get_color(),
                         ha='center', va='bottom', fontsize=9)

    plt.show()

# 그래프 실행
plot_content_preference_comparison(df_content_pref, order)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Matplotlib 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

# --- 1. 플랫폼별 이용률 성장률 데이터 (2023년 대비 2024년) ---
# 이전에 전처리한 플랫폼별 데이터를 기반으로 2024년 성장률을 추출 (예시 값)
data_platform_growth = {
    '플랫폼': ['넷플릭스', '티빙', '웨이브', '쿠팡플레이', '디즈니플러스'],
    '이용률_성장률_2024': [14.29, 27.69, 13.46, 3.39, 8.33] # (2023년 대비 2024년 성장률)
}
df_growth = pd.DataFrame(data_platform_growth)


# --- 2. 오리지널 콘텐츠 선호도 데이터 (전체 평균) ---
# OTT 서비스 전체 이용자의 오리지널 콘텐츠에 대한 5점 평균 선호도 (3.97점)를 기반으로
# 각 플랫폼의 오리지널 콘텐츠 경쟁력을 나타내는 지표를 가정 (예시 값)
# 이 지표는 해당 플랫폼의 오리지널 콘텐츠에 대한 평가가 높을수록 높은 값
data_orig_pref = {
    '플랫폼': ['넷플릭스', '티빙', '웨이브', '쿠팡플레이', '디즈니플러스'],
    '오리지널_콘텐츠_경쟁력_점수': [4.5, 3.8, 3.3, 3.6, 4.0] # (5점 만점 기준, 경쟁력 가정치)
}
df_pref = pd.DataFrame(data_orig_pref)


# --- 3. 데이터 통합 ---
df_combined = pd.merge(df_growth, df_pref, on='플랫폼')def plot_growth_vs_preference(df):
    plt.figure(figsize=(10, 7))

    # Scatter Plot: 오리지널 콘텐츠 경쟁력 vs. 이용률 성장률
    ax = sns.scatterplot(
        data=df,
        x='오리지널_콘텐츠_경쟁력_점수',
        y='이용률_성장률_2024',
        hue='플랫폼', # 플랫폼별로 색상 구분
        size='이용률_성장률_2024', # 성장률이 높을수록 점 크기를 키움
        sizes=(100, 500), # 점 크기 범위 설정
        palette='Set1'
    )

    plt.title('플랫폼별 이용률 성장률 (2024) vs. 오리지널 콘텐츠 경쟁력', fontsize=16, pad=20)
    plt.xlabel('오리지널 콘텐츠 경쟁력 점수 (가정치, 5점 만점)', fontsize=12)
    plt.ylabel('이용률 성장률 (%)', fontsize=12)

    # 데이터 레이블 (플랫폼 이름) 추가
    for line in range(0, df.shape[0]):
         ax.text(df['오리지널_콘텐츠_경쟁력_점수'][line]+0.02,
                 df['이용률_성장률_2024'][line],
                 df['플랫폼'][line],
                 horizontalalignment='left',
                 size='small',
                 color='black',
                 weight='semibold')

    plt.legend(loc='lower right', title='플랫폼')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

# 그래프 실행
plot_growth_vs_preference(df_combined)